In [1]:
# Cell 1: imports y configuración
import os
import pandas as pd
from sqlalchemy import create_engine

DB_USER = "model_user"
DB_PASS = "model_password"
DB_HOST = "mysql-service"
DB_PORT = 3306
RAW_DB  = "RawData"

RAW_URI = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{RAW_DB}"
engine  = create_engine(RAW_URI)

BATCH_SIZE = 15000

# Base de datos destino donde guardas los datos procesados
CLEAN_DB = "CleanData"
CLEAN_URI = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{CLEAN_DB}"

In [2]:
# Cell X: Limpieza total de RawData y verificación

from sqlalchemy import create_engine, inspect, text
import os

# Motores ya definidos antes:
# RAW_URI   = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{RAW_DB}"
# CLEAN_URI = …

engine_raw = create_engine(RAW_URI)
insp_raw   = inspect(engine_raw)

# Listamos todas las tablas en RawData
raw_tables = insp_raw.get_table_names()
print("Tablas en RawData antes de limpiar:", raw_tables)

# TRUNCATE cada tabla
with engine_raw.begin() as conn:
    for tbl in raw_tables:
        conn.execute(text(f"TRUNCATE TABLE `{tbl}`"))
        print(f"  → Vaciada tabla RawData.{tbl}")

# Verificamos que ahora cada tabla tenga 0 filas
print("\nVerificando conteos en RawData:")
with engine_raw.connect() as conn:
    for tbl in raw_tables:
        count = conn.execute(text(f"SELECT COUNT(*) FROM `{tbl}`")).scalar()
        print(f"  • {tbl}: {count} filas")

Tablas en RawData antes de limpiar: ['diabetes_data', 'diabetes_raw', 'staging_cleaned', 'staging_raw', 'staging_selected', 'test_data', 'train_data', 'train_pool', 'validation_data']
  → Vaciada tabla RawData.diabetes_data
  → Vaciada tabla RawData.diabetes_raw
  → Vaciada tabla RawData.staging_cleaned
  → Vaciada tabla RawData.staging_raw
  → Vaciada tabla RawData.staging_selected
  → Vaciada tabla RawData.test_data
  → Vaciada tabla RawData.train_data
  → Vaciada tabla RawData.train_pool
  → Vaciada tabla RawData.validation_data

Verificando conteos en RawData:
  • diabetes_data: 0 filas
  • diabetes_raw: 0 filas
  • staging_cleaned: 0 filas
  • staging_raw: 0 filas
  • staging_selected: 0 filas
  • test_data: 0 filas
  • train_data: 0 filas
  • train_pool: 0 filas
  • validation_data: 0 filas


In [3]:
# Cell Y: Limpieza total de CleanData y verificación
from sqlalchemy import create_engine, inspect, text

engine_clean = create_engine(CLEAN_URI)
insp_clean   = inspect(engine_clean)

# Listamos todas las tablas en CleanData
clean_tables = insp_clean.get_table_names()
print("Tablas en CleanData antes de limpiar:", clean_tables)

# TRUNCATE cada tabla
with engine_clean.begin() as conn:
    for tbl in clean_tables:
        conn.execute(text(f"TRUNCATE TABLE `{tbl}`"))
        print(f"  → Vaciada tabla CleanData.{tbl}")

# Verificamos que ahora cada tabla esté vacía
print("\nVerificando conteos en CleanData:")
with engine_clean.connect() as conn:
    for tbl in clean_tables:
        count = conn.execute(text(f"SELECT COUNT(*) FROM `{tbl}`")).scalar()
        print(f"  • {tbl}: {count} filas")

Tablas en CleanData antes de limpiar: ['diabetes_processed', 'diabetes_test_processed', 'diabetes_train_processed', 'diabetes_validation_processed']
  → Vaciada tabla CleanData.diabetes_processed
  → Vaciada tabla CleanData.diabetes_test_processed
  → Vaciada tabla CleanData.diabetes_train_processed
  → Vaciada tabla CleanData.diabetes_validation_processed

Verificando conteos en CleanData:
  • diabetes_processed: 0 filas
  • diabetes_test_processed: 0 filas
  • diabetes_train_processed: 0 filas
  • diabetes_validation_processed: 0 filas


In [4]:
DATA_ROOT    = "./data/Diabetes"
DATA_FILE    = os.path.join(DATA_ROOT, "Diabetes.csv")
GDRIVE_ID = "1k5-1caezQ3zWJbKaiMULTGq-3sz6uThC"
DOWNLOAD_URL = f"https://docs.google.com/uc?export=download&id={GDRIVE_ID}"
engine       = create_engine(RAW_URI)

In [5]:
def split_raw_data(test_size=0.2, val_size=0.25, random_state=42):
    """
    1) Descarga (si hace falta) y lee Diabetes.csv en un DataFrame
    2) Separa train_pool, validation_data, test_data
    3) Guarda esas tablas en RawData
    4) Crea train_data vacío para cargas incrementales
    """
    # — descargar y leer CSV —
    os.makedirs(DATA_ROOT, exist_ok=True)
    if not os.path.isfile(DATA_FILE):
        print("Descargando dataset…")
        r = requests.get(DOWNLOAD_URL, allow_redirects=True, stream=True)
        with open(DATA_FILE, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
        print("Descarga completada.")
    else:
        print("Dataset ya existe en disco.")
    
    print("Leyendo CSV con pandas…")
    df = pd.read_csv(DATA_FILE)
    print(f"Shape del DataFrame: {df.shape}")
    
    # — split: primero test —
    train_val, test = __import__('sklearn').model_selection.train_test_split(
        df, test_size=test_size, random_state=random_state
    )
    # — luego validación (val_size % de train_val) —
    train_pool, val = __import__('sklearn').model_selection.train_test_split(
        train_val, test_size=val_size, random_state=random_state
    )
    
    # — guardar en MySQL RawData —
    train_pool.to_sql("train_pool", engine, if_exists="replace", index=False)
    val.to_sql("validation_data", engine, if_exists="replace", index=False)
    test.to_sql("test_data", engine, if_exists="replace", index=False)
    
    # — crear train_data vacío para cargas incrementales —
    empty = train_pool.iloc[0:0]
    empty.to_sql("train_data", engine, if_exists="replace", index=False)
    
    print("Split completado:")
    print(f" • train_pool: {len(train_pool)} filas")
    print(f" • validation_data: {len(val)} filas")
    print(f" • test_data: {len(test)} filas")

In [6]:
def append_train_batch(batch_size=BATCH_SIZE):
    """
    Toma los primeros `batch_size` de train_pool que no estén aún en train_data,
    los añade a train_data. Ejecutar manualmente para simular llegada de datos.
    """
    df_pool = pd.read_sql("SELECT * FROM train_pool", engine)
    df_tr   = pd.read_sql("SELECT * FROM train_data", engine)

    # asume que existe una columna única (por ejemplo 'encounter_id')
    existing = set(df_tr['encounter_id'])
    new_rows = df_pool[~df_pool['encounter_id'].isin(existing)].head(batch_size)

    if new_rows.empty:
        print("No quedan nuevos registros en train_pool.")
        return

    new_rows.to_sql("train_data", engine, if_exists="append", index=False)
    print(f"Añadidos {len(new_rows)} registros a train_data.")


In [7]:
# Cell 2: Ejecutar el split inicial
split_raw_data()

Dataset ya existe en disco.
Leyendo CSV con pandas…
Shape del DataFrame: (101766, 50)
Split completado:
 • train_pool: 61059 filas
 • validation_data: 20353 filas
 • test_data: 20354 filas


In [8]:
# Cell 3: Ejecutar manualmente para añadir el primer batch de 15 000
append_train_batch()

Añadidos 15000 registros a train_data.


In [9]:
# Cell 4: Ver conteos actuales
for tbl in ["train_data","validation_data","test_data"]:
    cnt = pd.read_sql(f"SELECT COUNT(*) AS c FROM {tbl}", engine).iloc[0,0]
    print(f"{tbl}: {cnt} filas")

train_data: 15000 filas
validation_data: 20353 filas
test_data: 20354 filas


In [10]:
# Cell 3: Ejecutar manualmente para añadir el primer batch de 15 000
append_train_batch()

Añadidos 15000 registros a train_data.


In [11]:
# Cell 4: Ver conteos actuales
for tbl in ["train_data","validation_data","test_data"]:
    cnt = pd.read_sql(f"SELECT COUNT(*) AS c FROM {tbl}", engine).iloc[0,0]
    print(f"{tbl}: {cnt} filas")

train_data: 30000 filas
validation_data: 20353 filas
test_data: 20354 filas
